In [1]:
import notebook

import os.path, json, io, pandas
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (16, 20)

import pyspark.sql.functions as func # resuse as func.coalace for example
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType,DecimalType

%matplotlib inline

# make sure nbextensions are installed
notebook.nbextensions.check_nbextension('usability/codefolding', user=True)

try:
    sc
except NameError:
    import pyspark
    sc = pyspark.SparkContext('local[*]')
    sqlContext = pyspark.sql.SQLContext(sc)

In [2]:
import pandas as pd
import numpy as np

BASE_DIR             = os.path.join(os.path.abspath('.'), 'work-flow')
LASE_FILE            = os.path.join(BASE_DIR, 'test.csv')

rdd = sc.textFile(LASE_FILE).map(lambda x: [float(element) for element in x.split(",")])
df = sqlContext.createDataFrame(rdd)

## If small enough can be loaded firstly in pandas df
#pandas_df = pd.read_csv(LASE_FILE,  names = ['x','y','z'])
#df = sqlContext.createDataFrame(pandas_df)

In [3]:
print "Number of  Points: ", df.count()
df.printSchema()

Number of  Points:  214960
root
 |-- _1: double (nullable = true)
 |-- _2: double (nullable = true)
 |-- _3: double (nullable = true)



In [4]:
df.show()

+------------------+-----------------+--------------+
|                _1|               _2|            _3|
+------------------+-----------------+--------------+
| 5800905.972099781|545415.2531995773|104.7204971313|
| 5800905.984600067|545415.2567996979| 104.720703125|
| 5800905.995100021|545415.2587995529|104.7225036621|
| 5800906.049499989|545415.2775001526|104.7197036743|
| 5800906.064499855|545415.2827997208|104.7182006836|
|  5800906.07859993|545415.2876996994| 104.717300415|
| 5800906.011700153|545415.2660999298|104.7192001343|
| 5800906.022399902| 545415.267999649|104.7211990356|
| 5800906.037499905|545415.2741003036|  104.71900177|
|  5800905.83190012|545415.2767000198|104.7247009277|
| 5800905.841700077|545415.2782001495|104.7266998291|
|  5800905.85559988|545415.2838001251|104.7249984741|
| 5800905.867400169|545415.2870998383|104.7251968384|
| 5800905.879700184|  545415.29129982|104.7247009277|
| 5800906.120500088|545415.2311000824|104.7170028687|
| 5800906.134399891|545415.2

In [5]:
x0 = 5804500.0
y0 = 548500.0
r = 5.0

def get_cell_key(x,y,z):
    strx = int(np.floor( (x - x0) / r ))
    stry = int(np.floor( (y - y0) / r ))
    key = "cell_%s_%s" % (strx, stry)
    return key

# Test the function
value = [5800905.972099781, 545415.2531995773, 104.7204971313]
get_cell_key(*value)

'cell_-719_-617'

In [6]:
sqlContext.udf.register("get_cell_key", get_cell_key, StringType())

# http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.udf
# Creates a Column expression representing a user defined function (UDF).    
get_cell_key_udf = func.udf(get_cell_key, StringType())

ndf = df.select(
    get_cell_key_udf(df._1,df._2,df._3).alias('cell_id')
).groupBy("cell_id").count()

In [7]:
ndf.sort("count").show()

+--------------+-----+
|       cell_id|count|
+--------------+-----+
|cell_-721_-616|   13|
|cell_-719_-618|  194|
|cell_-721_-617| 1531|
|cell_-718_-616| 3146|
|cell_-718_-615| 4713|
|cell_-719_-617|26531|
|cell_-720_-615|27472|
|cell_-719_-615|30927|
|cell_-719_-616|33972|
|cell_-720_-616|42846|
|cell_-720_-617|43615|
+--------------+-----+

